In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from bs4 import BeautifulSoup
import codecs
import regex as re
import requests
from time import sleep
import time
import json
from collections import Counter

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common import action_chains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

PATH = "C:\chromedriver.exe"
options = Options()
options.add_argument('--no-sandbox')
options.add_argument("start-maximized")
options.add_argument("user-data-dir=C:\\Users\\alice\\AppData\\Local\\Google\\Chrome\\User Data\\Default")

## Define Functions

In [123]:
def get_unique_hrefs(site_name, min_char):
    """
    Write a function to get unique links 
    Return unique links and links that have many other links in it
    """
    hrefs = []
    misc_links = []
    misc = ['copyright', 'sitemap', 'contact', 'sponsor', 'legal', 'privacy-policy', 'plugins', 'wp-', 'resources']
    
    href = driver.find_elements_by_xpath("//*[@href]")
    for i in href: 
        try:
            i.get_attribute("href")
            hrefs.append(i.get_attribute("href"))
        except TimeoutException:
            driver.close()
            print('Timeout at this href %s ' % i)
            pass
        
    # Filtering for unique and nested links specific to the site name and minimum length of site name plus more
    links =[x for x in hrefs if x.startswith(site_name) and len(x) > min_char and not x.endswith('#')]
    
    # Have to find unique links this way not sure why "not in" didn't work
    for m in misc:
        for l in links:
            if m in l:
                misc_links.append(l)
    unique_misc_links = set(misc_links)
    final_non_misc_links = [link for link in links if link not in unique_misc_links]     
                
    return np.unique(final_non_misc_links)


def flatten(nested_lst):
    """
    Function to flatten nested list
    """
    flat_list = [item for sublist in nested_lst for item in sublist]
    
    return flat_list

def get_content(href, tag):
    """
    Function to get text of the blog 
    """
    content = []
    try:
#         print('I tried')
        vi = driver.find_element_by_xpath(tag).text
        content.append(vi)
    except NoSuchElementException:
        pass
    
#     except WebDriverException:
#         # For debugging purposes
#         print('WebDriver errs at %s ' % href)
#         driver.close()
#         driver = webdriver.Chrome(executable_path=PATH, options=options)
#         driver.get(href)
#         vi = driver.find_element_by_xpath(tag).text
#         content.append(vi)
    except:
        pass
    
    return content    

# Person 1: João Leitão
> Bio: My name is João Leitão and I was born in Lisbon – Portugal – back in 1980. My UN visited countries list goes up to more than 130. With a degree in Visual Arts, I like adventure travel, photography, and cooking.
Blog: https://www.joaoleitao.com/

## Get links

In [25]:
driver = webdriver.Chrome(executable_path=PATH, options=options)

site_name = "https://www.joaoleitao.com/"
url = "https://www.joaoleitao.com/adventure/"
driver.get(url)
hrefs_adventure = get_unique_hrefs(site_name, len(site_name))

In [29]:
hrefs_adventure_2 = []
for link in hrefs_adventure:
    driver = webdriver.Chrome(executable_path=PATH, options=options)
    driver.get(link)
    hrefs_adventure_2.append(get_unique_hrefs(site_name, len(site_name)))
    driver.close()

In [31]:
driver = webdriver.Chrome(executable_path=PATH, options=options)
url_2 = "https://www.joaoleitao.com/beautiful/"
driver.get(url_2)
hrefs_beautiful = (get_unique_hrefs(site_name, len(site_name)))

In [71]:
hrefs_adventure2_flatten = flatten(hrefs_adventure_2)

In [72]:
hrefs_adventure2_flatten.extend(hrefs_adventure)

In [73]:
hrefs_adventure2_flatten.extend(hrefs_beautiful)

In [74]:
len(hrefs_adventure2_flatten)

4353

In [76]:
all_hrefs = np.unique(hrefs_adventure2_flatten)

In [85]:
len(all_hrefs)

993

## Get content


In [6]:
all_hrefs = []
with open('saved_links_joao.txt', 'r') as f:
    for line in f:
        all_hrefs.append(line)
        
len(all_hrefs)

993

In [ ]:
all_content = []

for link in all_hrefs:
    driver = webdriver.Chrome(executable_path=PATH, options=options)
    driver.get(link)
    all_content.append(get_content(link, "//div[@class='entry-content']"))
    driver.close()

## Save content

In [14]:
with open('joao_blog_1.txt', 'w', encoding='utf-8') as f:
    for line in all_content:
        f.writelines('%s\n' % l for l in line)

In [12]:
all_content

[['Table of Contents\n5 Days in London\nTraveling and staying for 5 Days in London allows you to see all the city has to offer at a slow pace. You can even take the some time to explore a destination outside of the city with the number of possible day trips from London.\nOne week in London go by in the blink of an eye, but they’re good enough to experience the best out of the city. I’m going to help you plan your one week itinerary in London: Pubs, markets, museums, monuments, musicals, and so much more!\nThese are, of course, personal suggestions. The important thing is that you organize your trip according to your personal taste.\nOn this 5 days itinerary in London, you’ll find tips to visit the city by area, so you can get to (almost) everything on foot without wasting time getting around.\nOn this page, I’m sharing my step-by-step tips and travel advice with a day-by-day itinerary of London.\nHighlights\n1st\nDay\nSquare Mile\nMuseum of London\nLondon Wall and Aldersgate St.\nSt. P

# Person 2: Erin Loechner
> Bio: Author of Chasing Slow and founder of Other Goose, Erin Loechner has been blogging and speaking for more than a decade, garnering over one million fans worldwide. Her heartfelt writing and design work has been recognized in renowned international events for clients such as Walt Disney World, IKEA, Martha Stewart and Home Depot. Now nestled in a Midwestern town, Erin, her husband, and two kids strive for less in most areas except three: joy, grace, and goat cheese.  
To be scrapped up to:
https://designformankind.com/blog/page/850/  
Ground Truth:
https://designformankind.com/2014/09/faq/

## Get links

In [ ]:
site_name = "https://designformankind.com/"
erin_hrefs = []
page_range = np.arange(1,850)
for i in page_range:
    link = "https://designformankind.com/blog/page/" + str(i) + "/"
    driver = webdriver.Chrome(executable_path=PATH, options=options)
    driver.get(link)
    erin_hrefs.append(get_unique_hrefs(site_name, len(site_name)))
    driver.close()
# Left off at page 649

In [30]:
flatten_erin = flatten(erin_hrefs)

In [33]:
len(final_erins)

27185

In [85]:
seen = set()
uniq = [x for x in final_erins if x in seen or seen.add(x)]  

In [86]:
len(uniq)

19337

In [87]:
# save the links
with open('saved_links_erin.txt', 'w') as f:
    f.writelines('%s\n' % l for l in uniq)

In [88]:
# remove ground truth and reread
all_hrefs_erins = []
with open('saved_links_erin.txt', 'r') as f:
    for line in f:
        all_hrefs_erins.append(line)
        
len(all_hrefs_erins)

19337

## Get content

In [ ]:
# get content
all_content_erins = []
start_time = time.time()

for link in all_hrefs_erins[101:]:
    driver = webdriver.Chrome(executable_path=PATH, options=options)
    driver.get(link)
    all_content_erins.append(get_content(link, "//div[@class='entry-content jpibfi_container']"))
    time.sleep(3)
    driver.close()

end_time = time.time()
print('The total time getting content is %s '% np.round(end_time-start_time, 2))

## Save content

In [130]:
with open('erin_blog_the_rest.txt', 'w', encoding='utf-8') as f:
    for line in all_content_erins:
        f.writelines('%s\n' % l for l in line)

In [38]:
# Example of a few pages worth of text
all_content_erins

[['I am being interviewed, and the voice on the phone asks me why I’m not very social on social. It takes me a second to decode his words, or even his intent. Social on social? What?\nIs it not part of your strategy? he asks. You know, do you not see it as an essential part of your business plan?\nAnd well, let me just start there, because folks, there’s no business plan here.\nI know that’s terribly unpopular to admit. I know it’s not wise to say, that here I am flying by the seat of my pants, just hitting publish when the mood strikes. But I’d be lying if I told you otherwise.\nWant to hear something top-notch selfish?\nI blog for myself. It’s not a sales funnel, or a secret mission, or one small sliver of my thirty-year-plan. It’s just that I do it every morning because I’ve always done it every morning, and it changes me for the better.\nI write daily, in a chilly room with big windows a scratchy wool blanket, space heater at my feet. It is quiet. There is coffee.\nAnd that’s it. I